# PyCity Schools Analysis


*     Point #1 - Charter Schools had overall higher Math and Reading academic averages than District Schools. Analysis proves the overall percentage passing rate (based upon the average of the math and reading passing rates), was appr. 95% for Charter Schools and appr. 73% for District Schools. Thus, although charter schools have overall better passing rates than district schools and have overall smaller student body sizes, distinguishing amongst small and medium sized charter schools the level of achievement in math/reading based upon the student body size would not yield valuable a conclusion.


*     Point #2 - Schools with a greater amount of dollars available for spending did not significantly out-perform schools with less.

*     Point #3 - Based upon the breakdown of math scores by grade for each high school, it is observed that the math averages for each school do not vary a significant amount from grade level to grade level. 
There is a noticeable difference in the range for the averages between math and reading. For math, there are six high schools with C-average scores for math, Rodriguez High School with the approximately the lowest averages. However, the lowest reading averages do not trend below a score of 80. Although for certain high schools the math average was not significantly different from the reading averages, for those high schools which scored a C-average in math – their corresponding reading scores were noticeably higher and sat in the B-range. Overall, it can be observed that the reading average was consistently higher than the math average for each institution.

In [37]:
# Dependencies and Setup
import pandas as pd

# File to Load from Resources folder
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])

In [38]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [39]:
#Total Number of Schools
num_schools = school_data_df['School ID'].count()
print(f'There are {num_schools} schools in this district.')

#Total Number of Students
num_students = student_data_df['Student ID'].count()
print(f'There are {num_students} unique students in this district.')

#Total Budget
total_budget = school_data_df['budget'].sum()
print(f'The total budget of all {num_schools} schools is ${total_budget}.')

#Average Math Score 
avg_math_score = round(student_data_df['math_score'].mean(),2)
print(f'The average math score for all {num_students} students is {avg_math_score}.')

#Average Reading Score 
avg_reading_score = round(student_data_df['reading_score'].mean(),2)
print(f'The average reading score for all {num_students} students is {avg_reading_score}.')

#Overrall passing rate (overall average score) (avg math score + avg reading score/ 2)
passing_rate = round((avg_math_score + avg_reading_score)/2,2)
print(f'The overall passing rate is {passing_rate}')

#Percentage of Students with a passing math score (70 or greater)
perc_stu_math_pass = round(((student_data_df['math_score'] >= 70).sum() / student_data_df['math_score'].count())*100,2)
print(f'{perc_stu_math_pass}% of students received a math score of 70 or greater, which is passing.')


#Percentage of Students with a passing reading score (70 or greater)
perc_stu_read_pass = round(((student_data_df['reading_score'] >= 70).sum() / student_data_df['reading_score'].count())*100,2)
print(f'{perc_stu_read_pass}% of students received a reading score of 70 or greater, which is passing.')

#DataFrame to hold the above results
results_df = pd.DataFrame({"Total Schools": [num_schools],
                           "Total Students":[num_students],
                           "Total Budget":[total_budget],
                           "Avg Math Score":[avg_math_score],
                           "Avg Reading Score":[avg_reading_score],
                           "Passing Score(Overrall)":[passing_rate],
                           "Passing Rate(Math)(%)":[perc_stu_math_pass],
                           "Passing Rate(Reading)(%)":[perc_stu_read_pass]})

#Clean of results to display commas for thousands and improve dollar amounts
results_df["Total Students"] = results_df["Total Students"].map("{:,}".format)
results_df["Total Budget"] = results_df["Total Budget"].map("${:,.2f}".format)


results_df

There are 15 schools in this district.
There are 39170 unique students in this district.
The total budget of all 15 schools is $24649428.
The average math score for all 39170 students is 78.99.
The average reading score for all 39170 students is 81.88.
The overall passing rate is 80.44
74.98% of students received a math score of 70 or greater, which is passing.
85.81% of students received a reading score of 70 or greater, which is passing.


,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,Passing Score(Overrall),Passing Rate(Math)(%),Passing Rate(Reading)(%)
0,15,"39,170","$24,649,428.00",78.99,81.88,80.44,74.98,85.81


## School Summary

In [40]:
#Create overview tables by the school type and pertinent data;other table to calculate :budget;group by average
overview_table = school_data_df.filter(['school_name','type','size','budget'])
overview_table['budget/student'] = overview_table['budget']/overview_table['size']
sdc_grouped = school_data_complete.groupby(['school_name']).mean()

#Sort grouped data to analyze results of above
sdc_grouped = sdc_grouped.sort_index(ascending=True)
overview_table = overview_table.sort_values(by=['school_name'],ascending=True).set_index('school_name')

#Create tables by Math & Reading scores by school   
overview_table['AvgMathScore/School'] = sdc_grouped['math_score']
overview_table['AvgReadScore/School'] = sdc_grouped['reading_score']

# % Percentage Passing Math and Reading (70 or greater), and Overall Passing Rate
# Created two columns in school data complete DF returning T/F if student score is passing.
school_data_complete['num_pass_math'] = school_data_complete['math_score'] >= 70
school_data_complete['num_pass_read'] = school_data_complete['reading_score'] >= 70

# Group by the school name with function SUM to return number of True values (num of students who passed)
sdc_grouped_2 = school_data_complete.groupby(['school_name']).sum()

# Divide the number of students who passed by the total size of student population at each school
# New columns created in overview df to hold these values
overview_table['perc_pass_math'] = (sdc_grouped_2['num_pass_math'] / overview_table['size'])*100
overview_table['perc_pass_read'] = (sdc_grouped_2['num_pass_read'] / overview_table['size'])*100
overview_table['overall_pass_rate'] = (overview_table['perc_pass_math'] + overview_table['perc_pass_read']) / 2


# Clean Data Frame
school_summary_df = overview_table
school_summary_df = school_summary_df.rename(columns={"type":"School Type","size": "Total Students","budget": "Total School Budget","budget/student":"Per Student Budget",
                          "AvgMathScore/School":"Average Math Score","AvgReadScore/School":"Average Reading Score","perc_pass_math":"% Passing Math",
                         "perc_pass_read":"% Passing Reading","overall_pass_rate":"% Overall Passing Rate"})

school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,.2f}".format)



del school_summary_df.index.name
school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

In [41]:
# Sort and show top five schools
school_summary_df.sort_values(['% Overall Passing Rate'],ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [43]:
# Sort and show bottom five schools
school_summary_df.sort_values(['% Overall Passing Rate'],ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [44]:
#GROUP BY NINTH GRADERS, Avg Math Score
ninth_graders = student_data_df.loc[student_data_df['grade'] == '9th',:]
ninth_grader_series = pd.Series(ninth_graders['math_score'].values,index=ninth_graders['school_name'],name='9th')
ninth_grader_series = ninth_grader_series.groupby(['school_name']).mean()

#GROUP BY TENTH GRADERS, Avg Math Score
tenth_graders = student_data_df.loc[student_data_df['grade'] == '10th',:]
tenth_grader_series = pd.Series(tenth_graders['math_score'].values,index=tenth_graders['school_name'],name='10th')
tenth_grader_series = tenth_grader_series.groupby(['school_name']).mean()

#GROUP BY ELEVENTH GRADERS, Avg Math Score
eleventh_graders = student_data_df.loc[student_data_df['grade'] == '11th',:]
eleventh_grader_series = pd.Series(eleventh_graders['math_score'].values,index=eleventh_graders['school_name'],name='11th')
eleventh_grader_series = eleventh_grader_series.groupby(['school_name']).mean()

#GROUP BY TWELFTH GRADERS, Avg Math Score
twelfth_graders = student_data_df.loc[student_data_df['grade'] == '12th',:]
twelfth_grader_series = pd.Series(twelfth_graders['math_score'].values,index=twelfth_graders['school_name'],name='12th')
twelfth_grader_series = twelfth_grader_series.groupby(['school_name']).mean()

#Combined the four series into dataframe
all_gradersmath_df = pd.concat([ninth_grader_series,tenth_grader_series,eleventh_grader_series,twelfth_grader_series],axis = 1)
all_gradersmath_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [48]:
#GROUP BY NINTH GRADERS, Avg Reading Score
#(using ninth grader df already created - see math score by grade)
ninth_reading = pd.Series(ninth_graders['reading_score'].values,index=ninth_graders['school_name'],name='9th')
ninth_reading = ninth_reading.groupby(['school_name']).mean()

#GROUP BY TENTH GRADERS, Avg Reading Score
#(using tenth grader df already created - see math score by grade)
tenth_reading = pd.Series(tenth_graders['reading_score'].values,index=tenth_graders['school_name'],name='10th')
tenth_reading = tenth_reading.groupby(['school_name']).mean()

#GROUP BY ELEVENTH GRADERS, Avg Reading Score
#(using eleventh grader df already created - see math score by grade)
eleventh_reading = pd.Series(eleventh_graders['reading_score'].values,index=eleventh_graders['school_name'],name='11th')
eleventh_reading = eleventh_reading.groupby(['school_name']).mean()

#GROUP BY TWELFTH GRADERS, Avg Reading Score
#(using twelfth grader df already created - see math score by grade)
twelfth_reading = pd.Series(twelfth_graders['reading_score'].values,index=twelfth_graders['school_name'],name='12th')
twelfth_reading = twelfth_reading.groupby(['school_name']).mean()

#Combined 
all_grades_read_df = pd.concat([ninth_reading,tenth_reading,eleventh_reading,twelfth_reading],axis = 1)
all_grades_read_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [71]:
# Establish the bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Categorize the spending based on the bins
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(overview_table['budget/student'], spending_bins, labels=group_names)

spending_math_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_rate = (spending_math_scores + spending_reading_scores) / 2

# Assemble into data frame
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "% Passing Math": spending_passing_math,
                                 "% Passing Reading": spending_passing_reading,
                                 "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
spending_summary = spending_summary[["Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]]

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,83.694607
$585-615,83.599686,83.885211,94.230858,95.900287,83.742449
$615-645,79.079225,81.891436,75.668212,86.106569,80.485330
$645-675,76.997210,81.027843,66.164813,81.133951,79.012526


## Scores by School Size

In [72]:
# Establish the bins 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], size_bins, labels=group_names)

# Calculate the scores based on bins
size_math_scores = school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
overall_passing_rate = (size_passing_math + size_passing_reading) / 2

# Assemble into data frame
size_summary = pd.DataFrame({"Average Math Score" : size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_passing_math,
                             "% Passing Reading": size_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
size_summary = size_summary[["Average Math Score", 
                             "Average Reading Score", 
                             "% Passing Math", "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [74]:
# Type | Average Math Score | Average Reading Score | % Passing Math | % Passing Reading | % Overall Passing Rate

type_math_scores = school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
overall_passing_rate = (type_passing_math + type_passing_reading) / 2

# Assemble into data frame
type_summary = pd.DataFrame({"Average Math Score" : type_math_scores,
                             "Average Reading Score": type_reading_scores,
                             "% Passing Math": type_passing_math,
                             "% Passing Reading": type_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
type_summary = type_summary[["Average Math Score", 
                             "Average Reading Score",
                             "% Passing Math",
                             "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
